In [34]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import fiona
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = None 

# set working directory
os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')

In [35]:
# Load Data
combined_gdf = gpd.read_file('combined_gdf.geojson')

# Data Cleaning


In [36]:
print(combined_gdf.info())
print(combined_gdf.shape)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 26789 entries, 0 to 26788
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   trkg_num                  829 non-null    float64       
 1   Operator Name             829 non-null    object        
 2   facility_type             785 non-null    object        
 3   Spill_Desc                829 non-null    object        
 4   Spill Type                582 non-null    object        
 5   Root Cause                625 non-null    object        
 6   Preventative Measure      614 non-null    object        
 7   Root Cause Type           829 non-null    object        
 8   Detailed Root Cause Type  726 non-null    object        
 9   Long                      829 non-null    float64       
 10  Lat                       829 non-null    float64       
 11  facility_status           119 non-null    object        
 12  Metallic? 

In [37]:
combined_gdf.describe()

,trkg_num,Long,Lat,nearest_flowline_index,distance_m,CONSTRUCTDATE,Diam_in,ENDLAT,ENDLONG,FLOWLINEID,LOCATION_ID,Length_ft,MAXOPPRESSURE,OPERATOR_NUM,RECEIVE_DATE,SHAPE_Length,STARTLAT,STARTLOCATIONID,STARTLONG,risk
count,8.290000e+02,829.000000,829.000000,829.000000,829.000000,26789,26789.000000,26789.000000,26789.000000,26778.000000,26777.000000,26789.000000,15694.000000,26789.00000,26789,26789.000000,26789.000000,26456.000000,26789.000000,26789.000000
mean,4.027892e+08,-106.113582,39.904056,14475.650181,7882.204758,2006-06-16 05:38:45.237000,6.013843,40.282694,-104.237030,468241.731645,419830.374239,59910.929806,431.703645,54856.62018,2020-03-12 16:59:45.801000,18257.389225,40.282654,376715.146432,-104.237651,0.030946
min,4.015243e+08,-108.979884,37.014053,252.000000,0.032898,1900-01-11 00:00:00,0.000000,37.010428,-109.045542,455152.000000,159601.000000,2.200000,0.000000,710.00000,2018-04-25 14:38:45.767000,0.671472,36.993609,159652.000000,-109.049983,0.000000
25%,4.021632e+08,-108.111562,39.544287,10121.000000,277.361189,2001-11-01 00:00:00,2.000000,40.103130,-104.794480,464982.000000,422016.000000,1124.040000,150.000000,10633.00000,2019-06-28 13:32:02.080000,343.464898,40.103663,320174.000000,-104.793971,0.000000
50%,4.029494e+08,-104.980390,40.073695,11073.000000,2439.215219,2010-11-01 00:00:00,3.000000,40.343300,-103.870830,465855.000000,436612.000000,3759.640000,150.000000,46685.00000,2019-10-11 09:43:25.197000,1145.913570,40.346052,338099.000000,-103.868601,0.000000
75%,4.033799e+08,-104.541670,40.350400,21728.000000,10052.057017,2017-06-08 00:00:00,12.375000,40.811009,-103.789850,473115.750000,461055.000000,141290.450000,550.000000,96155.00000,2020-11-18 13:44:52.847000,43053.037992,40.811044,435694.000000,-103.795350,0.000000
max,4.040505e+08,-102.083057,40.997350,26355.000000,86157.038918,2020-06-01 00:00:00,278.000000,40.992630,-102.045863,484080.000000,483552.000000,152023.900000,4700.000000,200077.00000,2023-11-17 11:11:43.017000,46342.379790,40.996000,484366.000000,-102.046467,1.000000
std,6.996862e+05,1.887900,0.690894,6670.762513,12211.490590,NaN,5.191879,0.742253,1.259619,5406.798569,57711.150262,68337.113812,606.724600,40680.95814,NaN,20824.306930,0.742209,60761.756889,1.259411,0.173173


### Get line age from construction date

In [38]:
# Verify CONSTRUCTDATE is datetime type; convert if necessary
combined_gdf['CONSTRUCTDATE'] = pd.to_datetime(combined_gdf['CONSTRUCTDATE'])

# Calculate line_age
# Today's date
today = pd.Timestamp.now()

# Calculate the difference in years
combined_gdf['line_age_yr'] = (today - combined_gdf['CONSTRUCTDATE']).dt.days / 365.25

# Display the updated GeoDataFrame to verify 'line_age' column
print(combined_gdf[['CONSTRUCTDATE', 'line_age_yr']])

      CONSTRUCTDATE  line_age_yr
0        1972-08-07    52.780287
1        2018-01-04     7.370294
2        2011-08-10    13.774127
3        2004-05-10    21.023956
4        1993-11-07    31.529090
...             ...          ...
26784    2001-10-02    23.627652
26785    2008-02-22    17.237509
26786    2007-08-05    17.787817
26787    1998-09-27    26.642026
26788    1983-10-21    41.577002

[26789 rows x 2 columns]


In [39]:
# Print the list of column names
column_names = combined_gdf.columns.tolist()

print(column_names)

['trkg_num', 'Operator Name', 'facility_type', 'Spill_Desc', 'Spill Type', 'Root Cause', 'Preventative Measure', 'Root Cause Type', 'Detailed Root Cause Type', 'Long', 'Lat', 'facility_status', 'Metallic?', 'incident_date', 'nearest_flowline_index', 'matched_geometry', 'distance_m', 'ACTIONDESCRIPTION', 'BEDDINGMATERIAL', 'COMPANY_NAME', 'CONSTRUCTDATE', 'Diam_in', 'ENDLAT', 'ENDLONG', 'ENTIRELINEREMOVED', 'FLOWLINEACTION', 'FLOWLINEID', 'Fluid', 'LOCATIONTYPE', 'LOCATION_ID', 'Length_ft', 'MAXOPPRESSURE', 'Material', 'OPERATOR_NUM', 'Operator', 'PIPEMATERIAL', 'RECEIVE_DATE', 'SHAPE_Length', 'STARTLAT', 'STARTLOCATIONID', 'STARTLONG', 'Status', 'TYPEOFFLUIDTRANS', 'unique_id', 'risk', 'geometry', 'line_age_yr']


In [40]:
mapping = {
    'KINDER MORGAN CO2 CO LP': 'KINDER MORGAN CO2 CO LLC',
    'BEEMAN OIL & GAS INC': 'BEEMAN OIL & GAS LLC',
}
combined_gdf['Operator'] = combined_gdf['Operator'].replace(mapping)

combined_gdf.rename(columns={'OPERATOR_NUM': 'operator_number'}, inplace=True)
combined_gdf.rename(columns={'Operator': 'operator_name'}, inplace=True)

# Extract unique OPERATOR_NUM values
unique_operator_nums = combined_gdf['operator_number'].unique()

# Create a DataFrame to map OPERATOR_NUM to Operator (assuming the column name for operator names is 'Operator')
combined_gdf_operator_mapping = combined_gdf[['operator_number', 'operator_name']].drop_duplicates().reset_index(drop=True)

# Display the mapping
print(combined_gdf_operator_mapping)

     operator_number                        operator_name
0              10110  GREAT WESTERN OPERATING COMPANY LLC
1              69175                       PDC ENERGY INC
2              47120      KERR MCGEE OIL & GAS ONSHORE LP
3             100322                     NOBLE ENERGY INC
4              10459             EXTRACTION OIL & GAS INC
..               ...                                  ...
115            65110        O'BRIEN ENERGY RESOURCES CORP
116            41550        TYLER ROCKIES EXPLORATION LTD
117            10506               SEELEY OIL COMPANY LLC
118            11001                  BROWN OIL & GAS LLC
119            10639            CPX PICEANCE HOLDINGS LLC

[120 rows x 2 columns]


In [41]:
# List of columns to be removed
columns_to_remove = [
    'trkg_num', 'Operator Name', 'facility_type', 'Spill_Desc', 'Spill Type', 'Root Cause', 'Preventative Measure', 'Detailed Root Cause Type', 
    'Long', 'Lat', 'facility_status', 'Metallic?', 'nearest_flowline_index', 'ACTIONDESCRIPTION', 'BEDDINGMATERIAL', 'COMPANY_NAME', 
    'ENDLAT', 'ENDLONG', 'ENTIRELINEREMOVED', 'PIPEMATERIAL', 'RECEIVE_DATE', 'STARTLAT', 'STARTLOCATIONID', 'STARTLONG', 'TYPEOFFLUIDTRANS', 'operator_name', 'SHAPE_Length', 'distance_m', 'matched_geometry']

# Drop the columns from the DataFrame
combined_gdf = combined_gdf.drop(columns=columns_to_remove)

In [42]:
combined_gdf.head()

,Root Cause Type,incident_date,CONSTRUCTDATE,Diam_in,FLOWLINEACTION,FLOWLINEID,Fluid,LOCATIONTYPE,LOCATION_ID,Length_ft,MAXOPPRESSURE,Material,operator_number,Status,unique_id,risk,geometry,line_age_yr
0,Unknown,2018-01-23 00:00:00,1972-08-07,2.0,None,470450.0,Oil,Production Facilities,470449.0,542.71,NaN,Steel,10110,Active,flowline_5162,1,"LINESTRING (545287.5 4410654.506, 545132.965 4...",52.780287
1,Unknown,2018-02-08 00:00:00,2018-01-04,3.5,Registration,477981.0,Produced Fluids,Production Facilities,447490.0,404.27,NaN,Carbon Steel,69175,New Construction,flowline_5631,1,"LINESTRING (529087.406 4468617.814, 529165.786...",7.370294
2,Unknown,2018-03-09 00:00:00,2011-08-10,2.0,Abandonment,457300.0,PRODUCED WATER,Production Facilities,318070.0,18.23,NaN,Steel,47120,ACTIVE,flowline_14531,1,"LINESTRING (526647.996 4445503.586, 526647.873...",13.774127
3,Unknown,2018-03-15 00:00:00,2004-05-10,2.0,Registration,457931.0,Multiphase,Production Facilities,422528.0,1135.36,NaN,Carbon Steel,100322,Active,flowline_14519,1,"LINESTRING (534932.646 4463662.371, 535082.761...",21.023956
4,Unknown,2018-03-22 00:00:00,1993-11-07,2.0,Registration,466186.0,Multiphase,Production Facilities,455178.0,768.11,NaN,Carbon Steel,100322,Abandoned,flowline_11319,1,"LINESTRING (521484.139 4483010.433, 521636.926...",31.529090


In [43]:
# Reorder df
new_order = ['unique_id', 'operator_number', 'FLOWLINEID', 'LOCATION_ID', 'Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 'Fluid',
             'Material', 'Diam_in', 'Length_ft', 'MAXOPPRESSURE', 'line_age_yr', 'CONSTRUCTDATE', 'incident_date','Root Cause Type','geometry']
combined_gdf = combined_gdf[new_order]
combined_gdf

,unique_id,operator_number,FLOWLINEID,LOCATION_ID,Status,FLOWLINEACTION,LOCATIONTYPE,Fluid,Material,Diam_in,Length_ft,MAXOPPRESSURE,line_age_yr,CONSTRUCTDATE,incident_date,Root Cause Type,geometry
0,flowline_5162,10110,470450.0,470449.0,Active,None,Production Facilities,Oil,Steel,2.0,542.71,NaN,52.780287,1972-08-07,2018-01-23 00:00:00,Unknown,"LINESTRING (545287.5 4410654.506, 545132.965 4..."
1,flowline_5631,69175,477981.0,447490.0,New Construction,Registration,Production Facilities,Produced Fluids,Carbon Steel,3.5,404.27,NaN,7.370294,2018-01-04,2018-02-08 00:00:00,Unknown,"LINESTRING (529087.406 4468617.814, 529165.786..."
2,flowline_14531,47120,457300.0,318070.0,ACTIVE,Abandonment,Production Facilities,PRODUCED WATER,Steel,2.0,18.23,NaN,13.774127,2011-08-10,2018-03-09 00:00:00,Unknown,"LINESTRING (526647.996 4445503.586, 526647.873..."
3,flowline_14519,100322,457931.0,422528.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,2.0,1135.36,NaN,21.023956,2004-05-10,2018-03-15 00:00:00,Unknown,"LINESTRING (534932.646 4463662.371, 535082.761..."
4,flowline_11319,100322,466186.0,455178.0,Abandoned,Registration,Production Facilities,Multiphase,Carbon Steel,2.0,768.11,NaN,31.529090,1993-11-07,2018-03-22 00:00:00,Unknown,"LINESTRING (521484.139 4483010.433, 521636.926..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26784,flowline_14918,100322,455232.0,446052.0,Abandoned,Registration,Production Facilities,Multiphase,Carbon Steel,3.0,1413.74,NaN,23.627652,2001-10-02,None,None,"LINESTRING (530328.842 4472004.142, 529900.46 ..."
26785,flowline_14919,100322,455190.0,455177.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,2.0,527.60,NaN,17.237509,2008-02-22,None,None,"LINESTRING (543138.306 4481306.544, 542978.929..."
26786,flowline_14920,100322,455155.0,455096.0,Abandoned,Registration,Production Facilities,Multiphase,Carbon Steel,3.0,2070.49,NaN,17.787817,2007-08-05,None,None,"LINESTRING (519355.342 4482133.153, 519296.77 ..."
26787,flowline_14921,100322,455156.0,455096.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,2.0,800.31,NaN,26.642026,1998-09-27,None,None,"LINESTRING (519465.329 4482537.89, 519317.553 ..."


### Consolidate variables uniformly

In [44]:
# List of specific columns for which to print unique values
columns_to_check = ['Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 'Fluid', 'Material']

# Create a dictionary to store unique values for each column
unique_values_dict = {}

# Loop through the specified columns and store unique values in the dictionary
for column in columns_to_check:
    if column in combined_gdf.columns:  # Check if the column exists in the DataFrame
        unique_values = combined_gdf[column].unique().tolist()  # Convert numpy array to list
        unique_values_dict[column] = unique_values
    else:
        unique_values_dict[column] = "Column not found in DataFrame."

# Print each column's unique values list
for column, values in unique_values_dict.items():
    print(f"Unique values in {column}: {values}")

Unique values in Status: ['Active', 'New Construction', 'ACTIVE', 'Abandoned', 'REMOVED', 'ABANDONED', 'Out of Service', 'Out Of Service', 'Future', 'abandoned', 'ABiP', 'Abandoned in Place', 'Inactive', 'Pre-Abandonment', 'InActive', 'OOS', 'Shut in', 'Actove', 'Out of service', 'Removed', 'OutofService', 'Pre Abandonment', 'Avtive', 'shut in', 'PA', 'INACTIVE', 'Status', 'Out-of-Service', 'Actve', 'active', 'Abandon', 'PreAbandonment', 'TA', 'Abadnon', 'SI']
Unique values in FLOWLINEACTION: [None, 'Registration', 'Abandonment', 'Abandonment Verification', 'Out of Service', 'Pre-Abandonment Notice', 'Realignment', 'Removed From Service']
Unique values in LOCATIONTYPE: ['Production Facilities', 'Manifold', 'Well Site', 'Pit', 'Compressor Station', 'Gathering Line', 'Produced Water Transfer System', 'Crude Oil Transfer Line']
Unique values in Fluid: ['Oil', 'Produced Fluids', 'PRODUCED WATER', 'Multiphase', 'Natural Gas', 'Natural Gas Lift', '3 Phase', 'Produced Water', 'CO2', 'Natural 

In [45]:
status_mapping = {
    'Active': 'Active', 'ACTIVE': 'Active', 'Actove': 'Active', 'Avtive': 'Active', 'Actve': 'Active', 'active': 'Active',
    'Out of Service': 'Out of Service', 'OOS': 'Out of Service', 'OutofService': 'Out of Service', 'Out-of-Service': 'Out of Service', 'Out Of Service': 'Out of Service', 'Out of service': 'Out of Service',
    'Abandoned': 'Abandoned', 'abandoned': 'Abandoned', 'Abandoned in Place': 'Abandoned', 'ABANDONED': 'Abandoned', 'Abandon': 'Abandoned','Abadnon': 'Abandoned','TA': 'Abandoned',
    'Inactive': 'Inactive', 'InActive': 'Inactive', 'INACTIVE': 'Inactive',
    'PA': 'Pending Analysis', 'ABiP':'Pending Analysis', 'Shut in': 'Shut In', 'shut in': 'Shut In','SI':'Shut In',
    'Status': 'Unknown', 'Future': 'Future',
    'REMOVED': 'Removed',
    'Pre Abandonment':'Pre-Abandonment', 'PreAbandonment': 'Pre-Abandonment'
}
combined_gdf['Status'] = combined_gdf['Status'].replace(status_mapping)
print(combined_gdf['Status'].unique())


flowlineaction_mapping = {
    'Out of Service': 'Out of Service', 'Removed From Service': 'Out of Service',
    'Pre-Abandonment Notice': 'Pre-Abandonment Notice',
    'Abandonment Verification': 'Abandonment',
    'Realignment': 'Realignment',
    'Registration': 'Registration',
    'Abandonment': 'Abandonment'
}
combined_gdf['FLOWLINEACTION'] = combined_gdf['FLOWLINEACTION'].replace(flowlineaction_mapping)
print(combined_gdf['FLOWLINEACTION'].unique())


locationtype_mapping = {
    'Production Facilities': 'Production Facilities', 'Well Site': 'Well Site', 'Manifold': 'Manifold',
    'Compressor Station': 'Compressor Station', 'Gathering Line': 'Gathering Line',
    'Crude Oil Transfer Line': 'Crude Oil Transfer Line', 'Produced Water Transfer System': 'Produced Water Transfer System'
}
combined_gdf['LOCATIONTYPE'] = combined_gdf['LOCATIONTYPE'].replace(locationtype_mapping)
print(combined_gdf['LOCATIONTYPE'].unique())

['Active' 'New Construction' 'Abandoned' 'Removed' 'Out of Service'
 'Future' 'Pending Analysis' 'Inactive' 'Pre-Abandonment' 'Shut In'
 'Unknown']
[None 'Registration' 'Abandonment' 'Out of Service'
 'Pre-Abandonment Notice' 'Realignment']
['Production Facilities' 'Manifold' 'Well Site' 'Pit' 'Compressor Station'
 'Gathering Line' 'Produced Water Transfer System'
 'Crude Oil Transfer Line']


In [46]:
# Update the 'Fluid' column normalization and mapping in one comprehensive block
combined_gdf['Fluid'] = combined_gdf['Fluid'].str.strip().str.title().replace({
    'Natual Gas': 'Natural Gas',  # Correct common misspelling
    'Natural Gas Production': 'Natural Gas',
    'Co2': 'Co2/Produced Water',  # Assuming Co2 implies the mixed type
    'C02/Prod Water': 'Co2/Produced Water',
    'Co2/Prod Water': 'Co2/Produced Water',
    'Co2Produced Water': 'Co2/Produced Water',
    'Co2/Produced Wtaer': 'Co2/Produced Water',
    'Gas': 'Natural Gas',  # Assuming general Gas to be Natural Gas
    'Gas, Oil And Water': 'Full Well Stream',
    'Oil': 'Crude Oil',  # Assuming Oil to be categorized as Crude Oil
    'Crude Oil': 'Crude Oil',
    'Crude Oil Emulsion': 'Crude Oil Emulsion',
    'Emulsion': 'Crude Oil Emulsion',  # To unify with Crude Oil Emulsion
    'Crude Oil Emmulsion, Water And Oil': 'Crude Oil Emulsion',
    'Crude Oil And Water Emulsion': 'Crude Oil Emulsion',
    'Oil Water Emulsion': 'Crude Oil Emulsion',
    'Oil/Water': 'Crude Oil Emulsion',
    'Oil Water': 'Crude Oil Emulsion',
    'Oil And Water': 'Crude Oil Emulsion',
    'Oil /Water/Gas': 'Full Well Stream',
    'Oil/Gas/Water': 'Full Well Stream',
    'Oil, Gas, Water': 'Full Well Stream',
    '3 Phase': 'Multiphase',  # Assumed to mean the same
    'Multiphase': 'Multiphase',
    'Multi-Phase': 'Multiphase',
    'Mulitphase': 'Multiphase',
    'Multi Phase': 'Multiphase',
    'Mulit Phase': 'Multiphase',
    'Multi-Phase\xa0': 'Multiphase',  # Non-breaking space issue
    'Injection Produced Water': 'Produced Water',
    'Produced Water': 'Produced Water',
    'Water': 'Produced Water',
    'Saltwater': 'Produced Water',
    'Condensate': 'Condensate',  # Depending on your classification needs
    'Liquid': 'Other',
    'Liquids (Wtr/Cond)': 'Other', # General term, assuming to categorize broadly
    'Unprocessed Production Fluids': 'Other',
    'Production Fluids': 'Other',
    'Produced Fluids': 'Other',
    'Full Well Stream': 'Full Well Stream',  # Assuming similar handling
    'Other': 'Other',
    'Gas,  Oil And Water' : 'Full Well Stream',
    'Natural Gas Lift': 'Natural Gas',
    'Natuarl Gas': 'Natural Gas',
    'Natural Gas High Pressure': 'Natural Gas',
    'Natural Gas Supply':'Natural Gas',
    'Crude Oill Emulsion': 'Crude Oil Emulsion',
    'Unk': 'Unknown', 
    'Poly':'Polymer fluids'
})

# Review the changes to ensure the mappings are applied correctly
print(combined_gdf['Fluid'].unique())

['Crude Oil' 'Other' 'Produced Water' 'Multiphase' 'Natural Gas'
 'Co2/Produced Water' 'Crude Oil Emulsion' 'Condensate' 'Full Well Stream'
 'Polymer fluids' 'Unknown' 'Steel']


In [47]:
combined_gdf['Material'] = combined_gdf['Material'].str.strip().str.title().replace({
    'Fiberglass': 'Fiberglass', 
    'Fibergalss': 'Fiberglass', 
    'Fiberspar': 'Fiberglass', 
    'Fiber Glass': 'Fiberglass',
    'Carbon Steel': 'Carbon Steel', 
    'Carbonsteel': 'Carbon Steel',
    'Carbon Steel Sch 80': 'Carbon Steel',
    'Carbon Steel Sch 80': 'Carbon Steel',
    'Carbon Steel - Hdpe': 'Carbon Steel/HDPE', 
    'Carbon Steel, Hdpe,Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel, Hdpe, Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel',
    'Carbon Steel/Stainless Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/Hdpe/Stainless': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/Hdpe': 'Carbon Steel/HDPE', 
    'Satinless/Carbon Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Carbon Steel/Stainless/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Steel': 'Steel', 
    'Lined Steel': 'Steel', 
    'Coated Steel': 'Steel', 
    'Flexsteel': 'Steel', 
    'Flexpipe' : 'Steel',
    'Fiber Glass And Carbon Steel': 'Fiberglass/Carbon Steel', 
    'Fiberglass And Hdpe': 'Fiberglass/HDPE',
    'Hdpe': 'HDPE', 
    'Hdpe Poly': 'HDPE', 
    'Composite Hdpe': 'HDPE',
    'Hdpe/Steel': 'HDPE/Steel', 
    'Hdpe Lined Steel': 'HDPE/Steel',
     'Hdpe/Steel, Flexsteel': 'HDPE/Steel',
    'Poly': 'Polycarbonate', 
    'Polyline': 'Polycarbonate', 
    'Poly & Steel': 'Polycarbonate/Steel', 
    'Steel/Poly': 'Polycarbonate/Steel',
    'Poly/Steel': 'Polycarbonate/Steel',
    'Polycarbonate': 'Polycarbonate', 
    'Polycarbonate/Steel': 'Polycarbonate/Steel',
    'Pvc': 'PVC', 
    'Flexspar': 'Fiberglass', 
    'Stainless': 'Steel', 
    'Stainless/Carbon Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Carbon Steel/Hdpe/Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel',
    'Unknown': 'Unknown', 
    'Other': 'Other', 
    'Other (Poly)': 'Polycarbonate', 
    'Sdr7 Polyethelyne': 'Polyethylene', 
    'Sdr 11 Poly Pipe': 'Polyethylene', 
    'Sdr 11 Poly': 'Polyethylene', 
    'Poly Pipe': 'Polyethylene', 
    'Sdr_Poly': 'Polyethylene',
    'Poly': 'Polyethylene',
    'Poly Sdr 7': 'Polypropylene',
    'Poly Sdr-7': 'Polypropylene',
    'Duplex': 'Duplex', 
    'Fplp': 'Other', 
    'Flowline': 'Other',
    'Flex Steel': 'Steel',
    'Other (Flex Steel)': 'Steel',
    'Fiberglass And Carbon Steel': 'Carbon Steel/Fiberglass', 
    'Stainless Steel': 'Steel',
    'HDPE Lined Steel': 'HDPE/Steel',
    'Fiberglass/Hdpe': 'Fiberglass/HDPE',
    'Unk': 'Unknown', 'Other (Unknown)': 'Unknown', 'Other': 'Unknown',
})

# Verify the changes by printing the unique values in the 'Material' column
print(combined_gdf['Material'].unique())

['Steel' 'Carbon Steel' 'Polyethylene' 'Fiberglass' 'HDPE' 'Other'
 'Polycarbonate' 'PVC' 'Carbon Steel/HDPE/Stainless Steel'
 'Carbon Steel/HDPE' 'Unknown' 'HDPE/Steel' 'Duplex' 'Fiberglass/HDPE'
 'Polycarbonate/Steel' 'Polypropylene' 'Co2/Produced Water'
 'Carbon Steel/Fiberglass' 'Oil']


In [48]:
# Define the mapping dictionary for the 'Root Cause Type' column
root_cause_mapping = {
    'Corrosion': 'Corrosion', 
    'Unknown': 'Unknown', 
    'Incorrect Operation': 'Incorrect Operation', 
    'Equipment Failure': 'Equipment Failure', 
    'Equipment failure': 'Equipment Failure',  
    'Other Outside Force Damage': 'Other Outside Force Damage', 
    'Natural Force Damage': 'Natural Force Damage', 
    'Pipe, Weld, or Joint Failure': 'Pipe, Weld, or Joint Failure', 
    'Pipe, Weld Joint Failure': 'Pipe, Weld, or Joint Failure',
    'Excavation Damage': 'Excavation Damage', 
    'Other Outside Force': 'Other Outside Force Damage',  # Mapping to a more standardized category
    'Pipe, Weld, Joint Failure': 'Pipe, Weld, or Joint Failure'  # Ensuring consistency in naming
}

# Apply the mapping to the 'Root Cause Type' column
combined_gdf['Root Cause Type'] = combined_gdf['Root Cause Type'].replace(root_cause_mapping)
print(combined_gdf['Root Cause Type'].unique())

['Unknown' 'Corrosion' 'Equipment Failure' 'Incorrect Operations'
 'Pipe, Weld, or Joint Failure' 'Other Outside Force Damage'
 'Natural Force Damage' 'Excavation Damage' None]


In [49]:
combined_gdf.columns = [
    'unique_id',
    'operator_number',  # Changed from OPERATOR_NUM to operator_number
    'flowline_id',       # Changed from FLOWLINEID to flowline_id
    'location_id',       # Changed from LOCATION_ID to location_id
    'status',            # Already appropriately named but changed to lowercase
    'flowline_action',   # Changed from FLOWLINEACTION to flowline_action
    'location_type',     # Changed from LOCATIONTYPE to location_type
    'fluid',             # Already appropriately named but changed to lowercase
    'material',          # Already appropriately named but changed to lowercase
    'diameter_in',       # Changed from Diam_in to diameter_in
    'length_ft',         # Changed from Length_ft to length_ft
    'max_operating_pressure',
    'line_age_yr',
    'construct_date',
    'spill_date',
    'root_cause',
    'geometry',
    ]

# Print new column names to verify
print(combined_gdf.columns.tolist())

['unique_id', 'operator_number', 'flowline_id', 'location_id', 'status', 'flowline_action', 'location_type', 'fluid', 'material', 'diameter_in', 'length_ft', 'max_operating_pressure', 'line_age_yr', 'construct_date', 'spill_date', 'root_cause', 'geometry']


In [50]:
combined_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 26789 entries, 0 to 26788
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   unique_id               26789 non-null  object        
 1   operator_number         26789 non-null  int32         
 2   flowline_id             26778 non-null  float64       
 3   location_id             26777 non-null  float64       
 4   status                  26789 non-null  object        
 5   flowline_action         17803 non-null  object        
 6   location_type           26789 non-null  object        
 7   fluid                   26789 non-null  object        
 8   material                26789 non-null  object        
 9   diameter_in             26789 non-null  float64       
 10  length_ft               26789 non-null  float64       
 11  max_operating_pressure  15694 non-null  float64       
 12  line_age_yr             26789 non-null

In [51]:
combined_gdf.shape

(26789, 17)

In [52]:
# Adding the 'risk' column based on whether 'root_cause' is present or not
combined_gdf['risk'] = combined_gdf['root_cause'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [53]:
# Remove rows where 'max_operating_pressure' is NaN and 'risk' is 0
combined_gdf = combined_gdf[~((combined_gdf['max_operating_pressure'].isna()) & (combined_gdf['risk'] == 0))]

In [54]:
combined_gdf.shape

(16132, 18)

In [55]:
# Select the relevant numeric columns for KNN Imputation
columns_for_imputation = ['max_operating_pressure', 'diameter_in', 'length_ft', 'line_age_yr', 'material', 'fluid']  # Add other relevant columns as needed

# Creating a copy of the data to prevent modifying the original directly
df_for_imputation = combined_gdf[columns_for_imputation].copy()

# Convert the 'material' column using Label Encoding
le = LabelEncoder()
df_for_imputation['material_encoded'] = le.fit_transform(df_for_imputation['material'])
df_for_imputation['fluid_encoded'] = le.fit_transform(df_for_imputation['fluid'])

# Drop the original 'material' and 'fluid' columns
df_for_imputation = df_for_imputation.drop(columns=['material', 'fluid'])

# Initialize the KNN Imputer
imputer = KNNImputer(n_neighbors=5)

# Fit and transform the data to impute missing values
df_imputed = imputer.fit_transform(df_for_imputation)

# Replace the original max_operating_pressure with the imputed values
combined_gdf['max_operating_pressure'] = df_imputed[:, 0]

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [56]:
# make variables integers
combined_gdf['diameter_in'] = combined_gdf['diameter_in'].astype(int)
combined_gdf['length_ft'] = combined_gdf['length_ft'].astype(int)
combined_gdf['max_operating_pressure'] = combined_gdf['max_operating_pressure'].astype(int)
combined_gdf['line_age_yr'] = combined_gdf['line_age_yr'].astype(int)

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodatafr

In [57]:
combined_gdf

,unique_id,operator_number,flowline_id,location_id,status,flowline_action,location_type,fluid,material,diameter_in,length_ft,max_operating_pressure,line_age_yr,construct_date,spill_date,root_cause,geometry,risk
0,flowline_5162,10110,470450.0,470449.0,Active,None,Production Facilities,Crude Oil,Steel,2,542,34,52,1972-08-07,2018-01-23 00:00:00,Unknown,"LINESTRING (545287.5 4410654.506, 545132.965 4...",1
1,flowline_5631,69175,477981.0,447490.0,New Construction,Registration,Production Facilities,Other,Carbon Steel,3,404,2020,7,2018-01-04,2018-02-08 00:00:00,Unknown,"LINESTRING (529087.406 4468617.814, 529165.786...",1
2,flowline_14531,47120,457300.0,318070.0,Active,Abandonment,Production Facilities,Produced Water,Steel,2,18,2030,13,2011-08-10,2018-03-09 00:00:00,Unknown,"LINESTRING (526647.996 4445503.586, 526647.873...",1
3,flowline_14519,100322,457931.0,422528.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,2,1135,435,21,2004-05-10,2018-03-15 00:00:00,Unknown,"LINESTRING (534932.646 4463662.371, 535082.761...",1
4,flowline_11319,100322,466186.0,455178.0,Abandoned,Registration,Production Facilities,Multiphase,Carbon Steel,2,768,1006,31,1993-11-07,2018-03-22 00:00:00,Unknown,"LINESTRING (521484.139 4483010.433, 521636.926...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26662,flowline_14794,96155,456386.0,433999.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,3,1404,150,6,2018-06-13,None,None,"LINESTRING (597095.529 4518130.982, 597416.31 ...",0
26663,flowline_14795,96155,456381.0,433999.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,3,1404,150,6,2018-06-16,None,None,"LINESTRING (597095.529 4518130.982, 597416.31 ...",0
26664,flowline_14796,96155,456382.0,433999.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,3,1404,150,6,2018-06-13,None,None,"LINESTRING (597095.529 4518130.982, 597416.31 ...",0
26770,flowline_14902,35080,455592.0,443145.0,Active,Registration,Production Facilities,Crude Oil,HDPE,3,1175,40,7,2017-11-15,None,None,"LINESTRING (637207.682 4380630.737, 637458.657...",0


In [58]:
# Function to count only NaN values, excluding None
def count_only_nan(series):
    return series.apply(lambda x: 1 if isinstance(x, float) and np.isnan(x) else 0).sum()

# Apply the function to each column to count NaNs, not None
na_columns = combined_gdf.apply(count_only_nan)

# Filter columns with NaN values
columns_with_only_nan = na_columns[na_columns > 0]

# Print the results
print("Columns with NaN values (excluding None) and their counts:")
print(columns_with_only_nan)


Columns with NaN values (excluding None) and their counts:
flowline_id    2
location_id    9
dtype: int64


In [59]:
# Identify columns with NaN values
columns_with_na = combined_gdf.columns[combined_gdf.isna().any()]

# Create a dictionary to store the counts of NaNs with risk of 1 for each column
na_with_risk_1 = {}

# Iterate over each column with NaN values and count rows with risk = 1
for column in columns_with_na:
    count = combined_gdf[combined_gdf[column].isna() & (combined_gdf['risk'] == 1)].shape[0]
    na_with_risk_1[column] = count

# Print the counts of NaNs with risk of 1 for each column
print("Number of NaNs with risk of 1 in each column:")
print(na_with_risk_1)

Number of NaNs with risk of 1 in each column:
{'flowline_id': 0, 'location_id': 0, 'flowline_action': 148, 'spill_date': 0, 'root_cause': 0}


In [60]:
# Replace NaN with None in flowline_action for rows where risk == 1
combined_gdf.loc[
    (combined_gdf['risk'] == 1) & (combined_gdf['flowline_action'].isna()),
    'flowline_action'
] = 'None'

In [61]:
# combined_gdf = combined_gdf.dropna(columns=['flowline_action'])

In [62]:
combined_gdf = combined_gdf.dropna(subset=['flowline_id', 'location_id'])

In [63]:
combined_gdf


,unique_id,operator_number,flowline_id,location_id,status,flowline_action,location_type,fluid,material,diameter_in,length_ft,max_operating_pressure,line_age_yr,construct_date,spill_date,root_cause,geometry,risk
0,flowline_5162,10110,470450.0,470449.0,Active,None,Production Facilities,Crude Oil,Steel,2,542,34,52,1972-08-07,2018-01-23 00:00:00,Unknown,"LINESTRING (545287.5 4410654.506, 545132.965 4...",1
1,flowline_5631,69175,477981.0,447490.0,New Construction,Registration,Production Facilities,Other,Carbon Steel,3,404,2020,7,2018-01-04,2018-02-08 00:00:00,Unknown,"LINESTRING (529087.406 4468617.814, 529165.786...",1
2,flowline_14531,47120,457300.0,318070.0,Active,Abandonment,Production Facilities,Produced Water,Steel,2,18,2030,13,2011-08-10,2018-03-09 00:00:00,Unknown,"LINESTRING (526647.996 4445503.586, 526647.873...",1
3,flowline_14519,100322,457931.0,422528.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,2,1135,435,21,2004-05-10,2018-03-15 00:00:00,Unknown,"LINESTRING (534932.646 4463662.371, 535082.761...",1
4,flowline_11319,100322,466186.0,455178.0,Abandoned,Registration,Production Facilities,Multiphase,Carbon Steel,2,768,1006,31,1993-11-07,2018-03-22 00:00:00,Unknown,"LINESTRING (521484.139 4483010.433, 521636.926...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26662,flowline_14794,96155,456386.0,433999.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,3,1404,150,6,2018-06-13,None,None,"LINESTRING (597095.529 4518130.982, 597416.31 ...",0
26663,flowline_14795,96155,456381.0,433999.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,3,1404,150,6,2018-06-16,None,None,"LINESTRING (597095.529 4518130.982, 597416.31 ...",0
26664,flowline_14796,96155,456382.0,433999.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,3,1404,150,6,2018-06-13,None,None,"LINESTRING (597095.529 4518130.982, 597416.31 ...",0
26770,flowline_14902,35080,455592.0,443145.0,Active,Registration,Production Facilities,Crude Oil,HDPE,3,1175,40,7,2017-11-15,None,None,"LINESTRING (637207.682 4380630.737, 637458.657...",0


In [64]:
# Calculate the total number of rows
total_rows = combined_gdf.shape[0]

# Count rows where risk is 1
risk_1_count = combined_gdf[combined_gdf['risk'] == 1].shape[0]

# Count rows where risk is 0
risk_0_count = combined_gdf[combined_gdf['risk'] == 0].shape[0]

# Print the results
print(f"Total number of rows: {total_rows}")
print(f"Total number of rows with risk = 1: {risk_1_count}")
print(f"Total number of rows with risk = 0: {risk_0_count}")

Total number of rows: 16121
Total number of rows with risk = 1: 829
Total number of rows with risk = 0: 15292


# Download Cleaned Data

In [65]:
combined_gdf.to_file("cleaned_gdf.geojson", driver='GeoJSON')

In [66]:
combined_gdf_operator_mapping.to_csv('operator_mapping.csv', index=False)